In this notebook, I will use a linear and tree models. 

The data set will be different from the one used in ARIMA models. An obersvation will containg last weeks expenses, incomes and balance. Hence, I need to compute the balance at each week.
(many other features can be computed, but I will stick to those, I've already spent a lot of time on previous notebooks)

I first split data into train/validate/test sets. Then I train models on train set, predict on validation set, then choose the one that have best results (MAPE and MAE). We can't use a cross validation here, since data should be kept in its chronological order. (This step ois meant to avoid overfitting)

Finally I apply it on the test set. 
I will then compare it to the ARIMA model chosen before.

ARIMA model was trained on 80% against opnly 60% of data for the ones used here. This was necessary to test them both on same data (20% last percents).

I will not spend time on looking for the best parameters here (grid search) due to lack of time. It is more meant to show the options we got.

In [146]:
from sklearn.linear_model import LinearRegression

import pandas as pd
import numpy as np

from typing import Tuple

from datetime import timedelta

import joblib

### Read data

In [147]:
df_accounts = pd.read_csv(
    "../data/accounts_enough_history.csv", parse_dates=["update_date"]
)
df_transactions = pd.read_csv(
    "../data/transactions_enough_history.csv", parse_dates=["date"]
)

In [157]:
df_incomes = df_transactions[df_transactions["amount"] > 0]
df_expenses = df_transactions[df_transactions["amount"] < 0]

### Remove outliers

In [158]:
def remove_outliers_year_aggregation(df: pd.DataFrame) -> pd.DataFrame:
    df["year"] = df["date"].dt.year

    df_month_expenses = (
        df.groupby(["account_id", "year"])["amount"].agg(["mean", "std"]).reset_index()
    )
    df = pd.merge(
        df,
        df_month_expenses,
        on=["account_id", "year"],
        how="left",
        sort=False,
    )
    df["outlier"] = df["amount"] > (df["mean"] + 3 * df["std"])
    return df

In [159]:
df_incomes = remove_outliers_year_aggregation(df=df_incomes)
df_incomes = df_incomes[~df_incomes["outlier"]]

df_expenses["amount"] = abs(df_expenses["amount"])
df_expenses = remove_outliers_year_aggregation(df=df_expenses)
df_expenses = df_expenses[~df_expenses["outlier"]]

<ipython-input-158-e646f9ddc972>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["year"] = df["date"].dt.year
<ipython-input-159-589335085b07>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_expenses["amount"] = abs(df_expenses["amount"])
<ipython-input-158-e646f9ddc972>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

### Week resampling

In [31]:
def _resample_per_week_unique_account(df: pd.DataFrame) -> pd.DataFrame:
    """Resample date into weeks for a unique account."""
    account_id = df["account_id"].iloc[0]

    temp_df = df.sort_values(
        by="date", ascending=True
    )[["date", "amount"]]

    temp_df = temp_df.set_index("date", drop=True)

    temp_df = (
        temp_df.resample("W-MON").sum().reset_index()
    )
    temp_df["account_id"] = account_id

    return temp_df


def resample_per_week(df: pd.DataFrame) -> pd.DataFrame:
    """Return a dataframe with resampled data into weeks."""
    transactions_dfs = [data for _, data in df.groupby("account_id")]

    return pd.concat(
        [
            _resample_per_week_unique_account(
                df=df_unique_account,
            )
            for df_unique_account in transactions_dfs
        ]
    ).reset_index(drop=True)

In [32]:
df_expenses = resample_per_week(df=df_expenses)
df_incomes = resample_per_week(df=df_incomes)

### Compute history balance

Since I have already removed outliers from data, the balance is not the real one, but only an estimation using only incomes and expenses kept.

In [37]:
df_expenses["amount"] = df_expenses["amount"] * (-1)

df_transactions = pd.concat([df_incomes, df_expenses], axis=0)

In [68]:
def _get_balance_unique_account(
    df_transactions_unique_account: pd.DataFrame, balance: float, account_id: int
) -> pd.DataFrame:
    """Compute the balance using transactions."""
    grouped_transactions = (
        df_transactions_unique_account.groupby(["account_id", "date"])[
            "amount"
        ]
        .sum()
        .reset_index()
        .sort_values(by=["date"], ascending=False)
    )
    grouped_transactions["balance"] = round(
        balance - grouped_transactions[
        "amount"
    ].cumsum(), 2
    )

    return grouped_transactions


def get_balance_data(df_accounts: pd.DataFrame, df_trans: pd.DataFrame) -> pd.DataFrame:
    """Return a dataframe with history accounts balance."""
    account_dfs = [data for _, data in df_trans.groupby("account_id")]

    return pd.concat(
        [
            _get_balance_unique_account(
                df_transactions_unique_account=df,
                balance=df_accounts[df_accounts["id"] == df["account_id"].iloc[0]][
                    "balance"
                ].iloc[0],
                account_id=df["account_id"].iloc[0],
            )
            for df in account_dfs
        ]
    ).reset_index(drop=True)

In [69]:
df_balance = get_balance_data(df_accounts=df_accounts, df_trans=df_transactions)

In [145]:
df_balance

,account_id,date,amount,balance
0,2,2020-01-20,0.00,3.52
1,2,2020-01-13,-56.00,59.52
2,2,2020-01-06,-73.85,133.37
3,2,2019-12-30,0.00,133.37
4,2,2019-12-23,-78.74,212.11
...,...,...,...,...
2012,49,2019-07-08,1.20,500.75
2013,49,2019-07-01,0.00,500.75
2014,49,2019-06-24,-9.17,509.92
2015,49,2019-06-17,0.00,509.92


In [84]:
df_week_transactions

,date,amount_expenses,account_id,amount_incomes
0,2017-05-29,-5.90,2,2300.0
1,2017-06-05,-2293.69,2,0.0
2,2017-06-12,-0.00,2,0.0
3,2017-06-19,-5.90,2,0.0
4,2017-06-26,-0.00,2,123.0
...,...,...,...,...
47236,2020-01-06,-10.80,49,0.0
47237,2020-01-06,-10.80,49,7700.0
47238,2020-01-06,-10.80,49,1560.0
47239,2020-01-06,-10.80,49,370.0


### Create final dataset 

In [94]:
df_week_transactions = pd.merge(df_expenses, df_incomes, on=["date", "account_id"], how="left", sort=False, suffixes=["_expenses", "_incomes"])


In [97]:
df_week_transactions.fillna(0, inplace=True)

In [99]:
df_week_transactions = pd.merge(df_week_transactions, df_balance[["date", "balance", "account_id"]], on=["account_id", "date"], how="left", sort=False)


In [100]:
df_week_transactions

,date,amount_expenses,account_id,amount_incomes,balance
0,2017-05-29,-5.90,2,2300.0,-9335.05
1,2017-06-05,-2293.69,2,0.0,-7040.95
2,2017-06-12,-0.00,2,0.0,-9334.64
3,2017-06-19,-5.90,2,0.0,-9334.64
4,2017-06-26,-0.00,2,123.0,-9340.54
...,...,...,...,...,...
2000,2019-12-09,-10.80,49,0.0,60.44
2001,2019-12-16,-0.00,49,0.0,49.64
2002,2019-12-23,-0.00,49,0.0,49.64
2003,2019-12-30,-0.00,49,0.0,49.64


In [102]:
def _create_windows_unique_account(
    df: pd.DataFrame, max_empty_weeks: int
) -> pd.DataFrame:
    """Removes windows that have more than `max_empty_weeks` empty weeks for a unique account."""
    # Expense
    for i in range(1, 13):
        df[f"previous_expense_{i}"] = df["amount_expenses"].shift(i, fill_value=0)
    for i in range(1, 13):
        df[f"previous_incomes_{i}"] = df["amount_incomes"].shift(i, fill_value=0)
    for i in range(1, 12):  # Only to 11, since the actual balance is the one from the beginning of the week
        df[f"previous_balance_{i}"] = df["amount_incomes"].shift(i, fill_value=0)   

    return df[(df[[col for col in df.columns if 'expense' in col]] == 0).sum(1) < max_empty_weeks]


def create_windows(df: pd.DataFrame, max_empty_weeks: int) -> pd.DataFrame:
    """Removes windows that have more than `max_empty_weeks` empty weeks."""
    transactions_dfs = [data for _, data in df.groupby("account_id")]

    return pd.concat(
        [
            _create_windows_unique_account(
                df=df_unique_account, max_empty_weeks=max_empty_weeks
            )
            for df_unique_account in transactions_dfs
        ]
    ).reset_index(drop=True)

In [105]:
df_final = create_windows(df=df_week_transactions, max_empty_weeks=6)
df_final.head(2)

,date,amount_expenses,account_id,amount_incomes,balance,previous_expense_1,previous_expense_2,previous_expense_3,previous_expense_4,previous_expense_5,...,previous_balance_2,previous_balance_3,previous_balance_4,previous_balance_5,previous_balance_6,previous_balance_7,previous_balance_8,previous_balance_9,previous_balance_10,previous_balance_11
0,2017-08-14,-180.0,2,0.0,-9147.16,-11.94,-0.00,-0.0,-5.9,-13.26,...,0.0,15.0,0.0,0.0,150.0,123.0,0.0,0.0,0.0,2300.0
1,2017-08-21,-375.9,2,530.0,-9327.16,-180.00,-11.94,-0.0,-0.0,-5.90,...,140.0,0.0,15.0,0.0,0.0,150.0,123.0,0.0,0.0,0.0


## Split TRAIN / VALIDATION / TEST

In [106]:
def split_train_validation_test(
    df: pd.DataFrame, ratio_train: float, ratio_validation: float
) -> Tuple[pd.DataFrame]:
    """
    Return a tuple of 3 DataFrames containing train, validation and test sets.
    The split must respect the chronological order.
    """
    dfs_train = []
    dfs_validation = []
    dfs_test = []

    for account_id in df["account_id"].unique():
        df_temp = (
            df[df["account_id"] == account_id]
            .sort_values(by="date")
            .reset_index(drop=True)
        )
        index_train, index_validation = int(len(df_temp) * ratio_train), int(
            len(df_temp) * (ratio_validation + ratio_train)
        )
        dfs_train.append(df_temp[:index_train])
        dfs_validation.append(df_temp[index_train:index_validation])
        dfs_test.append(df_temp[index_validation:])

    return (
        pd.concat(dfs_train, ignore_index=True),
        pd.concat(dfs_validation, ignore_index=True),
        pd.concat(dfs_test, ignore_index=True),
    )


df_train, df_validate, df_test = split_train_validation_test(
    df=df_final, ratio_train=0.6, ratio_validation=0.2
)

## Models

In [127]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, median_absolute_error
from sklearn.linear_model import Ridge, Lasso
from xgboost import XGBRegressor

def mean_absolute_percentage_error(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return (
        np.mean(
            np.abs((y_true - y_pred) / y_true)[
                np.isfinite(np.abs((y_true - y_pred) / y_true))
            ]
        )
        * 100
    )


In [111]:
y_train = df_train["amount_expenses"]
X_train = df_train.drop(["amount_expenses", "date", "amount_incomes", "account_id"], axis=1)

y_val = df_validate["amount_expenses"]
X_val = df_validate.drop(["amount_expenses", "date", "amount_incomes", "account_id"], axis=1)

In [112]:
lin_reg = LinearRegression().fit(X_train, y_train)

In [119]:
y_val_pred = lin_reg.predict(X_val)
median_absolute_error(y_val_pred, y_val)

303.3926222912919

By computing incomes and balance features and applying a simple linear regression , we obtain a median absolute error higher than the one obtained using ARIMA model. 


#### Ridge / Lasso

In [123]:
reg_ridge = Ridge(alpha=1.0)
reg_ridge.fit(X_train, y_train)
y_val_pred = reg_ridge.predict(X_val)
median_absolute_error(y_val_pred, y_val)

303.39262227295103

In [124]:
reg_lasso = Lasso(alpha=0.1)
reg_lasso.fit(X_train, y_train)
y_val_pred = reg_lasso.predict(X_val)
median_absolute_error(y_val_pred, y_val)

/Users/drissdebbaghnour/.pyenv/versions/3.8.2/envs/mansa_test/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 56179450.613364756, tolerance: 120829.15585266564
  model = cd_fast.enet_coordinate_descent(


303.3926574847005

In [128]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
y_val_pred = xgb.predict(X_val)
median_absolute_error(y_val_pred, y_val)

273.1235461425781

The XGB performs better. It would be appropriate to test other models and parameters (gridsearch), but due to lack of time I will go forward with a classic XGBoost and its defaults parameters.

- An other improvement would be to add other features:
    - moving average
    - datetime features (week number in the the month [1, 2, 3, 4])
    - statistical metrics (median, quantiles, std...)

### Apply model on test set 

In [130]:
y_test = df_test["amount_expenses"]
X_test = df_test.drop(["amount_expenses", "date", "amount_incomes", "account_id"], axis=1)

In [133]:
xgb = XGBRegressor()
#xgb.fit(pd.concat([X_train, X_val], axis=0), pd.concat([y_train, y_val], axis=0))
xgb.fit(X_train, y_train)
y_test_pred = xgb.predict(X_test)
median_absolute_error(y_test_pred, y_test)

299.4901519775391

In [165]:
X_train.columns

Index(['balance', 'previous_expense_1', 'previous_expense_2',
       'previous_expense_3', 'previous_expense_4', 'previous_expense_5',
       'previous_expense_6', 'previous_expense_7', 'previous_expense_8',
       'previous_expense_9', 'previous_expense_10', 'previous_expense_11',
       'previous_expense_12', 'previous_incomes_1', 'previous_incomes_2',
       'previous_incomes_3', 'previous_incomes_4', 'previous_incomes_5',
       'previous_incomes_6', 'previous_incomes_7', 'previous_incomes_8',
       'previous_incomes_9', 'previous_incomes_10', 'previous_incomes_11',
       'previous_incomes_12', 'previous_balance_1', 'previous_balance_2',
       'previous_balance_3', 'previous_balance_4', 'previous_balance_5',
       'previous_balance_6', 'previous_balance_7', 'previous_balance_8',
       'previous_balance_9', 'previous_balance_10', 'previous_balance_11'],
      dtype='object')

### Save the trained model

In [136]:
# save the model to disk
filename = 'xgb_model.sav'
joblib.dump(xgb, filename)
 
# load the model from disk
loaded_model = joblib.load(filename)
y_test_pred = loaded_model.predict(X_test)
median_absolute_error(y_test_pred, y_test)

299.4901519775391

In [166]:
df_final.empty

False

In [167]:
pd.DataFrame().empty

True